# Caculating F-Score of Semgrep and Bandit

In [11]:
import pandas as pd
df = pd.read_excel('labels.xlsx', sheet_name = 'ESB')
labels = pd.read_excel('labels.xlsx', sheet_name = 'Mr chekidet labels')

In [12]:
band = df['Vulline Bandit'].str.split(',').explode()
bandpreds = (band.index.astype(str) + '-' + band).values
bandpreds

array([nan, nan, '2-25(CF1)', '3-1(DC)', '3-15(CI)', '3-30(CI)',
       '3-30(PT)', '3-38(CF1)', '4-1(DC)', '4-22(CI)', '4-31(PT)',
       '4-31(CI)', '4-39(CF1)', '5-13(CI)', '6-1(DC)', '6-16(CI)', nan,
       nan, nan, nan, nan, '12-17(CF1)', '13-12(CI)', '14-15(CF1)',
       '15-1(DC)', '15-16(CI)', '15-21(CF1)'], dtype=object)

In [13]:
sem = df['Vulline semgrep'].str.split(',').explode()
sempreds = (sem.index.astype(str) + '-' + sem).values
sempreds

array([nan, nan, '2-25(CF1)', '3-38(CF1)', '3-15(CI)', '4-39(CF1)',
       '4-22(CI)', nan, '6-16(CI)', nan, nan, nan, nan, nan, '12-17(CF1)',
       nan, '14-15(CF1)', '14-11(XSS)', '15-21(CF1)', '15-16(CI)'],
      dtype=object)

In [14]:
exploded_labels = labels['Vulline'].str.split(',').explode()
dist_labels = (exploded_labels.index.astype(str) + '-' + exploded_labels).values
dist_labels

array([nan, '1-4(XSS)', '1-5(XSS)', '2-25(DT)', '3-14(CI)', '3-15(CI)',
       '3-38(DT)', '4-19(CI)', '4-21(CI)', '4-22(CI)', '4-39(DT)',
       '5-7(CI)', '5-9(CI)', '5-10(CI)', '5-11(CI)', '6-14(CI)',
       '6-16(CI)', nan, nan, nan, nan, nan, '12-10(XSS)', '12-12(XSS)',
       '12-17(DT)', '13-12(CI)', '13-14(CI)', '13-17(CI)', '14-6(XSS)',
       '14-11(XSS)', '14-15(DT)', '15-8(CI)', '15-9(CI)', '15-10(CI)',
       '15-15(CI)', '15-16(CI)', '15-17(CI)', '15-21(DT)'], dtype=object)

In [15]:
def my_rec(labels, preds):
    rec_samples = [x for x in preds if x in labels]
    return len(rec_samples)/len(labels)

def my_pre(labels, preds):
    rec_samples = [x for x in preds if x in labels]
    if len(preds) == 0:
        return 0
    return len(rec_samples)/len(preds)

In [16]:
def bench_it(labels, preds, attack = ''):
    labels = [x for x in labels if attack.lower() in x.lower()]
    preds = [x for x in preds if attack.lower() in x.lower()]
    pre = my_pre(labels, preds)
    rec = my_rec(labels, preds)
    if (pre + rec) == 0:
        return [pre, rec, 0]
    def f1(precision, recall):
        return 2 * (precision * recall) / (precision + recall)
    return [pre, rec, f1(pre, rec)]

# About DataSet

In [17]:
labels_dist = exploded_labels.str.extract('\((.*)\)')[0].value_counts()

In [18]:
def full_bench(preds):
    atts = list(labels_dist.index)
    atts.append('')
    sems = []
    for attack in atts:
        temp = bench_it(dist_labels, preds, attack)
        temp.extend([attack if attack != '' else 'all', labels_dist[attack] if attack != '' else sum(labels_dist)])
        sems.append(temp)
    df = pd.DataFrame(sems, columns = ['Precision', 'Recall', 'F1-Score', 'Attack', 'Support'])
    df = df.set_index('Attack')
    return df

In [19]:
full_bench(bandpreds)

AttributeError: 'float' object has no attribute 'lower'

In [ ]:
len(set('20, 22, 23, 36, 73, 74, 77, 78, 79, 88, 89, 90, 91, 93, 94, 95, 99, 113, 116, 117, 185, 186, 200, 209, 215, 221, 227, 252, 259, 284, 285, 287, 295, 311, 312, 315, 321, 326, 327, 328, 330, 338, 340, 344, 345, 347, 348, 352, 359, 377, 390, 396, 398, 400, 404, 405, 409, 441, 477, 485, 489, 497, 502, 522, 523, 532, 538, 552, 561, 563, 570, 571, 573, 581, 584, 601, 610, 611, 614, 628, 642, 643, 657, 664, 665, 668, 669, 670, 671, 674, 685, 687, 691, 693, 697, 703, 705, 706, 707, 710, 732, 754, 755, 772, 776, 798, 827, 829, 834, 913, 916, 918, 922, 943, 1236, 1333'.split()))

In [ ]:
link = 'https://semgrep.dev/p/default'
import requests
requests.get(link).text